In [2]:
def get_p(roe, profit_growth):
    """
    Calculate p based on e and g using the exponential relationship:
    p = 3^((roe + profit_growth) / 30)
    """
    reasonable_pb = 3 ** ((roe + profit_growth) / 30)
    return reasonable_pb

# Example usage:
roe = 24
profit_growth = 11
book_value = 263
reasonable_pb = get_p(roe, profit_growth)
print(f"For roe = {roe} and profit_growth = {profit_growth}, p = {reasonable_pb:.2f}")
print(f'reasonable_px - {reasonable_pb * book_value}')

For roe = 24 and profit_growth = 11, p = 3.60
reasonable_px - 947.5392576338663
